# Tarea 2 Métodos para Clasificación
---
## Integrantes: 
- *José Eduardo Caimapo* -  jose.caimapo.12@sansano.usm.cl - 201373573-8
- *Rodrigo Sepúlveda* - rodrigo.sepulveda.12@sansano.usm.cl - 201204766-8

# 2. Análisis de audios como datos brutos

En la siguiente sección se estudiarán...

In [1]:
# Imports 
import matplotlib.pyplot  as plt
import numpy              as np
import pandas             as pd

from scipy                          import signal
from scipy.io                       import wavfile
from sklearn.decomposition          import PCA
from sklearn.discriminant_analysis  import LinearDiscriminantAnalysis  as LDA
from sklearn.model_selection        import train_test_split
from sklearn.preprocessing          import StandardScaler

In [2]:
def clean_filename(fname, string):
    file_name = fname.split('/')[1]
    if file_name[:2] == '__':
        file_name=string + file_name
    return file_name

SAMPLE_RATE = 44100

def load_wav_file(name, path):
    s, b = wavfile.read(path+name)
    assert s == SAMPLE_RATE
    return b

In [4]:
def visualize_border(model, x_data, y_data, title='', legend=['Categoria1', 'Categoria 2']):
    fig = plt.figure(figsize=(12,6))
    classes = [0,1]
    colors = [plt.cm.cool(float(i)) for i in classes]
    plt.scatter(x_data[y_data==0][:,0], x_data[y_data==0][:,1], s=50, c=colors[0], label=legend[0])
    plt.scatter(x_data[y_data==1][:,0], x_data[y_data==1][:,1], s=50, c=colors[1], label=legend[1])
    plt.grid(True)
    if model != None:
        x_min, x_max = plt.xlim()
        y_min, y_max = plt.ylim()
        xx, yy = np.meshgrid(np.linspace(x_min, x_max, 200), np.linspace(y_min, y_max, 200))
        Z = model.predict_proba(np.c_[xx.ravel(), yy.ravel()])
        Z = Z[:,0].reshape(xx.shape)
        plt.contour(xx, yy, Z, 1, cmap=plt.cm.hot)
        plt.axis((x_min, x_max, y_min, y_max))
    plt.title(title)
    plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
    plt.show()



In [5]:
def visualize_heatmap(model, x_data, y_data, title=''):
    fig = plt.figure(figsize=(13,5))
    plt.grid(True)
    plt.scatter(x_data[:,0], x_data[:,1], s=30, c=y_train, cmap=plt.cm.autumn)
    x_min, x_max = plt.xlim()
    y_min, y_max = plt.ylim()
    xx, yy = np.meshgrid(np.linspace(x_min, x_max, 200), np.linspace(y_min, y_max, 200))
    Z = model.predict_proba(np.c_[xx.ravel(), yy.ravel()])
    Z = Z[:,0].reshape(xx.shape)
    hb = plt.hexbin(xx.ravel(), yy.ravel(), C=Z.ravel(),  cmap=plt.cm.plasma)
    plt.scatter(x_data[:,0], x_data[:,1], s=30, c=y_train, cmap=plt.cm.cool)
    plt.axis((x_min, x_max, y_min, y_max))
    cb = fig.colorbar(hb, aspect=12)
    cb.set_label('Probabilidad')
    plt.title(title)
    plt.show()

In [6]:
def visualize_border_interactive(param):
    model = train_model(param)
    visualize_border(model, x_train, y_train)
    visualize_heatmap(model, x_train, y_train)

## a) Construya un dataframe con los datos a analizar. Describa el dataset y determine cuántos registros hay por clase.

In [7]:
df = pd.read_csv('data/heartbeat-sounds/set_a.csv')

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 176 entries, 0 to 175
Data columns (total 4 columns):
dataset     176 non-null object
fname       176 non-null object
label       124 non-null object
sublabel    0 non-null float64
dtypes: float64(1), object(3)
memory usage: 3.5+ KB


In [9]:
df.head()

,dataset,fname,label,sublabel
0,a,set_a/artifact__201012172012.wav,artifact,NaN
1,a,set_a/artifact__201105040918.wav,artifact,NaN
2,a,set_a/artifact__201105041959.wav,artifact,NaN
3,a,set_a/artifact__201105051017.wav,artifact,NaN
4,a,set_a/artifact__201105060108.wav,artifact,NaN


In [10]:
df = df.drop(['dataset','sublabel'], axis=1)

In [11]:
df.head()

,fname,label
0,set_a/artifact__201012172012.wav,artifact
1,set_a/artifact__201105040918.wav,artifact
2,set_a/artifact__201105041959.wav,artifact
3,set_a/artifact__201105051017.wav,artifact
4,set_a/artifact__201105060108.wav,artifact


In [12]:
label_counts = df['label'].value_counts()
print label_counts

artifact    40
murmur      34
normal      31
extrahls    19
Name: label, dtype: int64


## b) Lea los archivos .wav y transformelos en secuencias de tiempo. Realice un padding de ceros al final de cada secuencia para que todas queden representadas con la misma cantidad de elementos, explique la importancia de realizar este paso.

In [13]:
def padd_zeros(array, length):
    aux = np.zeros(length)
    aux[:array.shape[0]] = array
    return aux

In [14]:
new_df = pd.DataFrame({'file_name' : df['fname'].apply(clean_filename, string='Aunlabelledtest')})
new_df['time_series'] = new_df['file_name'].apply(load_wav_file, path='data/heartbeat-sounds/set_a/')
new_df['len_series']  = new_df['time_series'].apply(len)
new_df['time_series'] = new_df['time_series'].apply(padd_zeros,length=max(new_df['len_series']))

In [15]:
new_df.head()

,file_name,time_series,len_series
0,artifact__201012172012.wav,"[1.0, -3.0, -1.0, -7.0, -9.0, -2.0, -6.0, -5.0...",396900
1,artifact__201105040918.wav,"[-2.0, 3.0, -4.0, 4.0, -3.0, 2.0, -1.0, 0.0, 0...",396900
2,artifact__201105041959.wav,"[6.0, -4.0, -9.0, -1.0, -4.0, 1.0, -5.0, 2.0, ...",396900
3,artifact__201105051017.wav,"[-85.0, -198.0, -214.0, -173.0, -177.0, -206.0...",396900
4,artifact__201105060108.wav,"[53.0, -35.0, 47.0, 170.0, 340.0, 436.0, 535.0...",396900


Si bien la importancia de este paso, no se aprecia aquí se ve más adelante que la normalización o incluso la concadenación por zero_padding aumenta la cantidad de puntos donde se aplica FFT produciendo de esta forma una mejor resolución en la frecuencia.

## c) Manipule los datos y cambie las etiquetas de los audios por otras asignadas por un doctor experto, el cual afirma que estos cambios son requeridos. Vuelva a determinar cuántos registros hay por clase. Nótese que ahora son 3 clases ¿Explique la problemática de tener etiquetas mal asignadas en los datos? ¿Un solo dato puede afectar esto?

In [16]:
new_labels = [
    0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
    0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1,
    1, 1, 1, 1, 1, 2, 1, 1, 1, 2, 2, 2, 1, 1, 2, 1, 2, 2, 1, 2, 2, 2, 2, 2, 2,
    2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1,
    1, 2, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 2, 2, 1, 1, 1, 1, 1, 2, 1, 0,
    2, 2, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 2, 1, 0, 1, 1, 1, 1, 1, 2, 0, 0, 0,
    0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2
]


labels = ['artifact', 'normal/extrahls', 'murmur']
new_df['target'] = [labels[i] for i in new_labels]

In [17]:
new_df.head()

,file_name,time_series,len_series,target
0,artifact__201012172012.wav,"[1.0, -3.0, -1.0, -7.0, -9.0, -2.0, -6.0, -5.0...",396900,artifact
1,artifact__201105040918.wav,"[-2.0, 3.0, -4.0, 4.0, -3.0, 2.0, -1.0, 0.0, 0...",396900,artifact
2,artifact__201105041959.wav,"[6.0, -4.0, -9.0, -1.0, -4.0, 1.0, -5.0, 2.0, ...",396900,artifact
3,artifact__201105051017.wav,"[-85.0, -198.0, -214.0, -173.0, -177.0, -206.0...",396900,artifact
4,artifact__201105060108.wav,"[53.0, -35.0, 47.0, 170.0, 340.0, 436.0, 535.0...",396900,artifact


In [18]:
new_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 176 entries, 0 to 175
Data columns (total 4 columns):
file_name      176 non-null object
time_series    176 non-null object
len_series     176 non-null int64
target         176 non-null object
dtypes: int64(1), object(3)
memory usage: 3.5+ KB


In [19]:
new_df['target'].value_counts()

normal/extrahls    65
artifact           58
murmur             53
Name: target, dtype: int64

En si tener mal aplicadas las etiquetas en los datos, implica que la máquina se condiciona y aprende a simular otro tipo de finalidad, que realmente no se tenia la intención.
Dependiendo de la sensibilidad de la máquina si esta es afectada por un solo dato sería un claro indicio de que la máquina clasificadora no aprendió la estructura general que los datos tienen (existiria un sobreajuste) dado a que sabe comportamientos muy particulares.

## d) Codifique las distintas clases a valores numéricos para que puedan ser trabajados por los algoritmos clasificadores.

In [20]:
new_df["target"] = new_df["target"].astype('category')
cat_columns = new_df.select_dtypes(['category']).columns
new_df[cat_columns] = new_df[cat_columns].apply(lambda x: x.cat.codes)

In [21]:
new_df.head()

,file_name,time_series,len_series,target
0,artifact__201012172012.wav,"[1.0, -3.0, -1.0, -7.0, -9.0, -2.0, -6.0, -5.0...",396900,0
1,artifact__201105040918.wav,"[-2.0, 3.0, -4.0, 4.0, -3.0, 2.0, -1.0, 0.0, 0...",396900,0
2,artifact__201105041959.wav,"[6.0, -4.0, -9.0, -1.0, -4.0, 1.0, -5.0, 2.0, ...",396900,0
3,artifact__201105051017.wav,"[-85.0, -198.0, -214.0, -173.0, -177.0, -206.0...",396900,0
4,artifact__201105060108.wav,"[53.0, -35.0, 47.0, 170.0, 340.0, 436.0, 535.0...",396900,0


In [22]:
new_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 176 entries, 0 to 175
Data columns (total 4 columns):
file_name      176 non-null object
time_series    176 non-null object
len_series     176 non-null int64
target         176 non-null int8
dtypes: int64(1), int8(1), object(2)
memory usage: 3.0+ KB


## e) Desordene los datos, evitando así el orden en el que vienen la gran mayoría de las etiquetas. Cree la matriz que conforma a los datos en sus dimensiones sin preprocesar, es decir, cada ejemplo es una secuencia de amplitudes en el tiempo. ¿Las dimensiones de ésta indica que puede generar problemas? ¿De qué tipo?

In [23]:
new_df = new_df.sample(frac=1, random_state=44)
x_data = np.stack(new_df['time_series'].values, axis=0)
y_data = new_df.target.values
x_data.shape

(176, 396900)

## f) Para pre procesar la secuencia en el tiempo realice una transformada de fourier discreta [5] para pasar los datos desde el dominio de tiempos al dominio de frecuencias presentes en la señal de sonido.

In [24]:
x_fourier = np.abs(np.fft.fft(x_data))

MemoryError: 

## g) Para seguir con el pre procesamiento realice un muestreo representativo de los datos a través de una técnica de muestreo especializada en secuencias ¿En qué beneficia este paso? ¿Cómo podría determinar si el muestro es representativo?

In [34]:
x_resampled = []

for i in range(x_fourier.shape[0]):
    sequence = x_fourier[i,:].copy()
    resampled_sequence = signal.resample(sequence, 100000)
    x_resampled.append(resampled_sequence)
    
x_resampled = np.array(x_resampled)
x_resampled.shape

NameError: name 'x_fourier' is not defined

## h) Genere un conjunto de pruebas mediante la técnica hold-out validation para verificar la calidad de los clasificadores. ¿Cuántas clases tiene y de qué tamaño queda cada conjunto?

In [28]:
x_train, x_test, y_train, y_test = train_test_split(x_resampled, y_data, test_size=0.25, random_state=42)

## i)  Realice un proceso de estandarizar los datos para ser trabajados adecuadamente. Recuerde que solo se debe ajustar (calcular media y desviación estándar) con el conjunto de entrenamiento.

In [29]:
std = StandardScaler(with_mean=True, with_std=True)
std.fit(x_train)
x_train = std.transform(x_train)
x_test = std.transform(x_test)

## j) Realice una reducción de dimensionalidad a través de la técnica PCA, para representar los datos en d = 2 dimensiones. Recuerde que solo se debe ajustar (encontrar las componentes principales) con el conjunto de entrenamiento. Visualice apropiadamente la proyección en 2 dimensiones.

In [30]:
d=2
pca_model = PCA(n_components=d)
pca_model.fit(x_train)
x_pca_train = pca_model.transform(x_train)
x_pca_test = pca_model.transform(x_test)

## k) Entrene un modelo de Regresión Logística variando el parámetro de regularizacion C construyendo un gráfico resumen del error en función de este hiper-parámetro. Además entrene una Máquina de Soporte Vectorial (SVM) con kernel lineal, variando el hiper-parámetro de regularizacion C en el mismo rango que para la Regresión Logística, construyendo el mismo gráfico resumen. Compare.

In [ ]:
Cs = [0.1,1,10,100,1000]
penalty = {'l1':'l1','l2':'l2'}
learner = {"SVM":"SVM", "Regresión logística":"LR"}
interactive(visualize_border_interactive, param = Cs, penalty = penalty, learner = learner)

## l) Entrene un Arbol de Decisión, con la configuración que estime conveniente, variando el hiper-parámetro regularizador max depth, construyendo un gráfico resumen del error en función de este parámetro. Compare con los modelos anteriores.

## m) Experimente con diferentes dimensiones d para la proyección de PCA con el propósito de obtener un modelo con menor error. Construya una tabla o gráfico resumen.

## n) Realice otra reducción de dimensionalidad ahora a través de la técnica LDA, para representar los datos en d = 2 dimensiones. Recuerde que sólo se debe ajustar con el conjunto de entrenamiento, si semmuestra un warning explique el porqué. Visualice apropiadamente la proyección en 2 dimensiones.

## o) Con el propósito de encontrar el mejor modelo vuelva a realizar el item h) con el i) en el nuevo espacio generado por la representaciçón según las d dimensiones de la proyección LDA. Esta nueva representación ¿mejora o empeora el desempeño? Explique.

## p) Intente mejorar el desempeño de los algoritmos ya entrenados. Diseñe ahora sus propias cracterísticas (feature crafting) a partir de los datos brutos (secuencia de amplitudes), puede inspirarse en otros trabajos si desea.